In [ ]:
import pandas as pd
import pandas_ta as ta
from datetime import datetime
import dask.dataframe as dd
from datetime import timedelta
import time
from multiprocessing import Pool
import itertools
import concurrent.futures
import threading
import dask
import dask.dataframe as dd
import dask.distributed as distributed
from dask import delayed
from dask.distributed import LocalCluster
from dask.distributed import Client
from dask.distributed import Client, progress
from dask.distributed import Client

In [ ]:
price = pd.read_csv(r"C:\Users\yaswa\Downloads\D.csv")
price = price.drop("Unnamed: 0",axis = 1)
price.head(10)

,Ticker,Date,Time,Open,High,Low,Close,Volume,Open Interest,datetime,otm_call,otm_put
0,BANKNIFTY-I.NFO,03/01/2022,09:15:59,36621.00,36621.00,35708.05,35811.85,82600,2347925,2022-01-03 09:15:59,36000,35700
1,BANKNIFTY-I.NFO,03/01/2022,09:16:59,35813.95,35825.75,35798.70,35798.70,38900,2347925,2022-01-03 09:16:59,35900,35600
2,BANKNIFTY-I.NFO,03/01/2022,09:17:59,35796.55,35796.90,35757.40,35782.60,28775,2368975,2022-01-03 09:17:59,35900,35600
3,BANKNIFTY-I.NFO,03/01/2022,09:18:59,35782.60,35834.25,35782.60,35834.25,28900,2368975,2022-01-03 09:18:59,36000,35700
4,BANKNIFTY-I.NFO,03/01/2022,09:19:59,35830.50,35839.65,35823.40,35835.00,28275,2368975,2022-01-03 09:19:59,36000,35700
5,BANKNIFTY-I.NFO,03/01/2022,09:20:59,35839.00,35840.00,35810.05,35810.60,25550,2383875,2022-01-03 09:20:59,36000,35700
6,BANKNIFTY-I.NFO,03/01/2022,09:21:59,35809.65,35809.65,35773.85,35785.85,20150,2383875,2022-01-03 09:21:59,35900,35600
7,BANKNIFTY-I.NFO,03/01/2022,09:22:59,35787.40,35797.00,35775.05,35794.05,19425,2383875,2022-01-03 09:22:59,35900,35600
8,BANKNIFTY-I.NFO,03/01/2022,09:23:59,35799.25,35799.25,35742.55,35752.90,32225,2393775,2022-01-03 09:23:59,35900,35600
9,BANKNIFTY-I.NFO,03/01/2022,09:24:59,35742.00,35742.00,35673.25,35698.35,44550,2393775,2022-01-03 09:24:59,35800,35500


In [ ]:
premiums = pd.read_csv(r"C:\Users\yaswa\Downloads\BankNifty.csv")
#premiums = premiums.drop("Unnamed: 0",axis = 1


In [ ]:
premiums['expiry_date'] = premiums['Ticker'].str[9:16]
premiums['strike_price'] = premiums['Ticker'].str[16:23]

In [ ]:
premiums = premiums[~premiums['expiry_date'].str.contains('-I.NFO|-II.NFO|-III.NF')]
premiums['expiry_date'] = pd.to_datetime(premiums['expiry_date'], format='%d%b%y')

In [ ]:
price["otm_call"] = price["otm_call"].astype(str)
price["otm_put"] = price["otm_put"].astype(str) 


In [ ]:
price["datetime"] = pd.to_datetime(price["datetime"])
premiums["datetime"] = pd.to_datetime(premiums["datetime"])

In [ ]:
price = price[["datetime","otm_call","otm_put"]]
price['otm_call'] = price['otm_call'].shift(periods=1)
price['otm_put'] = price['otm_put'].shift(periods=1)

In [ ]:
price["otm_call"] = price["otm_call"].astype(str)
price["otm_put"] = price["otm_put"].astype(str)

In [ ]:
type(price["otm_call"][1])

str

In [ ]:
price

,datetime,otm_call,otm_put
0,2022-01-03 09:15:59,None,None
1,2022-01-03 09:16:59,36000,35700
2,2022-01-03 09:17:59,35900,35600
3,2022-01-03 09:18:59,35900,35600
4,2022-01-03 09:19:59,36000,35700
...,...,...,...
91018,2022-12-28 15:26:59,42900,42600
91019,2022-12-28 15:27:59,43000,42700
91020,2022-12-28 15:28:59,43000,42700
91021,2022-12-28 15:29:59,42900,42600


In [ ]:
premiums = premiums[["Time","Open","High","Low","Close","datetime","expiry_date","strike_price"]]

In [ ]:
def implementation_Requirements(row,t):
   # y = pd.to_datetime(row['datetime'])- pd.Timedelta(minutes=1)
    date_str_new = row["datetime"] - timedelta(minutes = 1)

    e = Supertrend(t, row["otm_call"], row["otm_put"],date_str_new)
    #None, None,None,None,None,None
    #print(e)
    if e is None :
        return None, None,None,None,None,None
        
    r = t.loc[(t['strike_price'] == row["otm_call"] + "CE") & (t["datetime"] == date_str_new) & (t["expiry_date"] ==e[1]) ]
    if len(r) == 0:
        return None, None,None,None,None,None
    r_call = r['Close'].values[0]
    o = t.loc[(t['strike_price'] == row["otm_put"] + "PE") & (t["datetime"] == date_str_new) & (t["expiry_date"] == e[1])]
    if len(o) == 0:
        return None, None,None,None,None,None
    r_put = o['Close'].values[0] 
    u = t.loc[(t['strike_price'] == row["otm_call"] + "CE") & (t["datetime"] == row["datetime"]) & (t["expiry_date"] ==e[1])]
    if len(u) == 0:
        return None, None,None,None,None,None
    u_call = u['Open'].values[0]
    p = t.loc[(t['strike_price'] == row["otm_put"] + "PE") & (t["datetime"] == row["datetime"]) & (t["expiry_date"] == e[1])]
    if len(p) == 0:
        return None, None,None,None,None,None
    
    u_put = p['Open'].values[0]
    time_obj = row["datetime"].time()
    
    return r_call, r_put, u_call, u_put,time_obj,e

In [ ]:
premiums

,Time,Open,High,Low,Close,datetime,expiry_date,strike_price
0,09:15:59,270.90,270.90,241.50,250.00,2022-01-03 09:15:59,2022-01-27,34000PE
1,09:15:59,2.00,2.05,1.65,1.90,2022-01-03 09:15:59,2022-01-06,43000CE
2,09:15:59,723.90,723.90,723.90,723.90,2022-01-03 09:15:59,2022-01-13,36100PE
3,09:15:59,2.85,2.95,2.40,2.65,2022-01-03 09:15:59,2022-01-06,39000CE
4,09:15:59,563.90,590.00,561.05,590.00,2022-01-03 09:15:59,2022-01-27,35300PE
...,...,...,...,...,...,...,...,...
25173270,15:30:59,605.00,605.00,605.00,605.00,2022-12-28 15:30:59,2023-01-05,43200PE
25173271,15:30:59,105.00,105.00,105.00,105.00,2022-12-28 15:30:59,2023-01-25,45100CE
25173272,15:30:59,5.50,5.50,5.50,5.50,2022-12-28 15:30:59,2022-12-29,40700PE
25173273,15:30:59,90.05,90.05,90.05,90.05,2022-12-28 15:30:59,2023-01-05,41600PE


In [ ]:
o = price[50:]
o = o.reset_index(drop=True)
position = None
start_time = pd.Timestamp('2022-1-03 09:20:00')
timeframe = pd.Timedelta(minutes=5)
last_trade_exit_time = start_time - timeframe
time_str2 = "15:10:00"
time_str1 = "9:20:00"
time_str3 = "15:20:00"

time_obj1 = datetime.strptime(time_str1, '%H:%M:%S').time()
time_obj2 = datetime.strptime(time_str2, '%H:%M:%S').time()
time_obj3 = datetime.strptime(time_str3, '%H:%M:%S').time()
last_call = 0
last_put = 0
D = []

In [ ]:
o["datetime"][0]

Timestamp('2022-01-03 10:05:59')

In [ ]:
len(o)

90973

In [ ]:
price

,datetime,otm_call,otm_put
0,2022-01-03 09:15:59,None,None
1,2022-01-03 09:16:59,36000,35700
2,2022-01-03 09:17:59,35900,35600
3,2022-01-03 09:18:59,35900,35600
4,2022-01-03 09:19:59,36000,35700
...,...,...,...
91018,2022-12-28 15:26:59,42900,42600
91019,2022-12-28 15:27:59,43000,42700
91020,2022-12-28 15:28:59,43000,42700
91021,2022-12-28 15:29:59,42900,42600


In [ ]:
premiums

,Time,Open,High,Low,Close,datetime,expiry_date,strike_price
0,09:15:59,270.90,270.90,241.50,250.00,2022-01-03 09:15:59,2022-01-27,34000PE
1,09:15:59,2.00,2.05,1.65,1.90,2022-01-03 09:15:59,2022-01-06,43000CE
2,09:15:59,723.90,723.90,723.90,723.90,2022-01-03 09:15:59,2022-01-13,36100PE
3,09:15:59,2.85,2.95,2.40,2.65,2022-01-03 09:15:59,2022-01-06,39000CE
4,09:15:59,563.90,590.00,561.05,590.00,2022-01-03 09:15:59,2022-01-27,35300PE
...,...,...,...,...,...,...,...,...
25173270,15:30:59,605.00,605.00,605.00,605.00,2022-12-28 15:30:59,2023-01-05,43200PE
25173271,15:30:59,105.00,105.00,105.00,105.00,2022-12-28 15:30:59,2023-01-25,45100CE
25173272,15:30:59,5.50,5.50,5.50,5.50,2022-12-28 15:30:59,2022-12-29,40700PE
25173273,15:30:59,90.05,90.05,90.05,90.05,2022-12-28 15:30:59,2023-01-05,41600PE


In [ ]:
starttime = time.time()

## Dask

In [ ]:
import dask.bag as db
from dask import compute,delayed


Convert every code to Dask.

compute() function may seem costly, but multiprocessing through pandas dataframe is too difficult.

In [ ]:
def Supertrend(dataset, call_otm, put_otm, date_time,row,period = 10, multiplier=3):
    if call_otm is None:
        return None
    call = call_otm + "CE"
    put = put_otm + "PE"
    date_obj = row["datetime"].date()
    premiums_copy = dataset.copy()
    #premiums_copy['expiry_date'] = pd.to_datetime(premiums_copy['expiry_date'])
    #premiums_copy['datetime'] = pd.to_datetime(premiums_copy['datetime'])
    k_call = premiums_copy[((premiums_copy["datetime"]) <= date_time) & ((premiums_copy["strike_price"] == call)) & (premiums_copy['expiry_date'] >= pd.Timestamp(date_obj))]
    k_call = k_call.reset_index(drop=True)
    
    if len(k_call.index) == 0:
        return None
    
    a = k_call.sort_values(by='expiry_date')["expiry_date"].iloc[0]
    k_call = k_call[k_call['expiry_date'] == a]
    k_put = premiums_copy[((premiums_copy["datetime"]) <= date_time) & ((premiums_copy["strike_price"] == put)) & (premiums_copy['expiry_date'] == a)]
    k_put = k_put.reset_index(drop=True)
    
    if len(k_put.index) == 0:
        return None
    
    k_call = k_call[k_call["datetime"].isin(k_put["datetime"])]
    k_put = k_put[k_put["datetime"].isin(k_call["datetime"])]
    k_put = k_put.reset_index(drop=True)
    k_call = k_call.reset_index(drop=True)
    merged = pd.merge(k_call, k_put, on='datetime')
    merged['Open'] = merged['Open_x'] + merged['Open_y']
    merged['Close'] = merged['Close_x'] + merged['Close_y']
    merged['High'] = merged['High_x'] + merged['High_y']
    merged['Low'] = merged['Low_x'] + merged['Low_y']
    merged = merged[["Open", "Low", "Close", "High","datetime"]]
    merged['group'] = (merged.index // 5) + 1
    if len(merged) < 50:
        return None
    merged_resampled = merged.groupby('group').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})
    supertrend = ta.supertrend(high=merged["High"], low=merged["Low"], close=merged["Close"], period = 10, multiplier =3)
    # if "SUPERT_7_3.0" not in supertrend.columns:
        #return None
    merged_resampled["Supertrend"] = supertrend["SUPERT_7_3.0"]
    if len(merged) % 5 == 0:
        super_trend = merged_resampled["Supertrend"].iloc[-1]
    else:
        super_trend = merged_resampled["Supertrend"].iloc[-2]
        
    return [super_trend,a]


In [ ]:

def implementation_Requirements(row,t):
   # y = pd.to_datetime(row['datetime'])- pd.Timedelta(minutes=1)
    date_str_new = row["datetime"] - timedelta(minutes = 1)

    e = Supertrend(t, row["otm_call"], row["otm_put"],date_str_new,row)
    #None, None,None,None,None,None
    #print(e)
    if e is None :
        return None, None,None,None,None,None
        
    r = t.loc[(t['strike_price'] == row["otm_call"] + "CE") & (t["datetime"] == date_str_new) & (t["expiry_date"] ==e[1]) ]
    if len(r) == 0:
        return None, None,None,None,None,None
    r_call = r['Close'].values[0]
    o = t.loc[(t['strike_price'] == row["otm_put"] + "PE") & (t["datetime"] == date_str_new) & (t["expiry_date"] == e[1])]
    if len(o) == 0:
        return None, None,None,None,None,None
    r_put = o['Close'].values[0] 
    u = t.loc[(t['strike_price'] == row["otm_call"] + "CE") & (t["datetime"] == row["datetime"]) & (t["expiry_date"] ==e[1])]
    if len(u) == 0:
        return None, None,None,None,None,None
    u_call = u['Open'].values[0]
    p = t.loc[(t['strike_price'] == row["otm_put"] + "PE") & (t["datetime"] == row["datetime"]) & (t["expiry_date"] == e[1])]
    if len(p) == 0:
        return None, None,None,None,None,None
    
    u_put = p['Open'].values[0]
    time_obj = row["datetime"].time()
    
    return r_call, r_put, u_call, u_put,time_obj,e

In [ ]:
def process_date(date,df):
    #print(1)
    timeframe = pd.Timedelta(minutes=5)
    last_trade_exit_time =  pd.Timestamp('2022-1-03 09:20:00')
    time_str2 = "15:10:00"
    time_str1 = "9:20:00"
    time_str3 = "15:20:00"

    time_obj1 = datetime.strptime(time_str1, '%H:%M:%S').time()
    time_obj2 = datetime.strptime(time_str2, '%H:%M:%S').time()
    time_obj3 = datetime.strptime(time_str3, '%H:%M:%S').time()
    last_call = 0
    last_put = 0
    D = []
    group = price[price['datetime'].dt.date == date] 
    position = None
    for i, row in group.iterrows():
        t = df[(df["datetime"]<=row["datetime"]) & (df["datetime"] > (row["datetime"] - timedelta(days = 70)))]
        
    
    
        r_call, r_put, u_call, u_put,time_obj,e = implementation_Requirements(row,t)
        
        if  r_call is None:
            continue
    
    
    
    # check if it's time to scan for new trades
        if ((row['datetime']) >= (last_trade_exit_time + timeframe)) and (position is None) and  (time_obj <= time_obj2) and (time_obj >= time_obj1):
            if r_call + r_put < e[0]:
                position = "Entry"
            # = pd.concat([D, pd.DataFrame([[position,u_call,u_put,row["datetime"],row["otm_call"],row["otm_put"]]], columns=D.columns)], ignore_index=True)
            #print("ntry",i)
                last_call = row["otm_call"]
                last_put = row["otm_put"]
                D.append(["Entry",u_call,row["otm_call"],u_put,row["otm_put"],row["datetime"]])
         #D = D + [["Entry",u_call,row["otm_call"],u_put,row["otm_put"],row["datetime"]]]
        elif position is not None:
            if (time_obj <= time_obj3):
                if row["otm_call"] == last_call and row["otm_put"] == last_put:
                    if r_call + r_put > e[0]:
                    #D = D + [["Exit_Stoploss",u_call,last_call,last_put,row["otm_put"],row["datetime"]]]
                        D.append(["Exit_Stoploss",u_call,last_call,last_put,row["otm_put"],row["datetime"]])
                        position = None
                        last_trade_exit_time = row["datetime"]
                        last_call = 0
                        last_put = 0
                    
                   #D = pd.concat([D, pd.DataFrame([[position,u_call,u_put,row["datetime"],row["otm_call"],row["otm_put"]]], columns=D.columns)], ignore_index=True)
                    #print("Exit",i)
                    
                    
                else:
                    position = None
                #print("Exit","Change")
                    i= t.loc[(t['strike_price'] == last_call + "CE") & (t["datetime"] == row["datetime"]) & (t["expiry_date"] == e[1])]
                    i_call = i["Open"].values[0]
                    y= t.loc[(t['strike_price'] == last_put + "PE") & (t["datetime"] == row["datetime"]) & (t["expiry_date"] == e[1])]
                    i_put = y["Open"].values[0]
                    last_trade_exit_time = row["datetime"]
                #D = D + [["Exit_Change",i_call,last_call,i_put,last_put,row["datetime"]]]
                    D.append(["Exit_Change",i_call,last_call,i_put,last_put,row["datetime"]])
                    last_call = 0
                    last_put = 0
            else:
            #D = D+[["Exit_Squareoff",u_call,last_call,last_put,row["otm_put"],row["datetime"]]]
                D.append(["Exit_Squareoff",u_call,last_call,last_put,row["otm_put"],row["datetime"]])
                position = None
                last_trade_exit_time = row["datetime"]
                last_call = 0
                last_put = 0 
               #D = pd.concat([D, pd.DataFrame([[position,u_call,u_put,row["datetime"],row["otm_call"],row["otm_put"]]], columns=D.columns)], ignore_index=True)
            #print("Exit",i)  
            
            
    return pd.DataFrame(D)

In [ ]:
def premiums_last(date,df):
    date = pd.Timestamp(date)
    t = df[(df["datetime"]<= date + timedelta(days = 1)) & (df["datetime"] > (date - timedelta(days = 70)))]
    return t
    

Use delayed function for Parallelization of Code

In [ ]:
price

In [ ]:
dates[-1]

In [ ]:
%%time
process_date(dates[-1],premiums)

In [ ]:
process_date(dates[-1],premiums)

# Increasing the Limit of My Cluster.

In [ ]:
df = dd.from_pandas(premiums, npartitions=4)
df
type(df)

In [ ]:
df = df.repartition(npartitions=50)




In [ ]:
import dask.distributed

# create a Dask client
client = dask.distributed.Client()

# compute a Dask dataframe
df = dask.datasets.timeseries()

# scatter the Dask dataframe
df_scattered = client.scatter(df)

# submit a computation on the Dask dataframe
future = client.submit(lambda df: df.sum(), df_scattered)

# wait for the computation to complete and get the result
result = client.gather(future)

# print the result
print(result)


In [ ]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1,memory_limit='12GB')
client = Client(cluster)

In [ ]:
client

In [ ]:
client.close()

In [ ]:
df_scattered = client.scatter(df)

In [ ]:
results = []

In [ ]:
for date in k:
    result = client.submit(process_date, date, df_scattered)
    results.append(result)

In [ ]:
final_result = client.submit(pd.concat, results).result()

In [ ]:
import dask.dataframe as dd
import dask.distributed

# create a Dask client
client = dask.distributed.Client()

# create a Dask DataFrame

# scatter the Dask DataFrame to the workers
df_scattered = client.scatter(df)

# persist the Dask DataFrame in memory
df_persisted = client.persist(df_scattered)

# get the first partition of the Dask DataFrame
df_part = client.compute(df_persisted.partitions[0]).result()
print(df_part.head())

In [ ]:
df = client.scatter(df, timeout=150)

In [ ]:
dates[200]

In [ ]:
for client in Client._instances:
    client.close()

In [ ]:
Client._instances

In [ ]:
process_date(k[0],df)

In [ ]:

# Create futures for each date and the Dask DataFrame
date_futures = client.map(lambda date: process_date(date, df_scattered), k)

In [ ]:

#y = dask.distributed.default_client().scheduler_info()['workers'].keys()

In [ ]:
print(date_futures)

In [ ]:
results = client.gather(date_futures)

In [ ]:
for result in results:
    print(result.compute())

In [ ]:
type(client)

In [ ]:
print(results)

In [ ]:
client.close()

In [ ]:
status = client.status(date_futures)


In [ ]:
results = []
for date in k:
    result = process_date(date, df)
    results.append(result)


In [ ]:
computed_results = client.compute(results)

In [ ]:
print(computed_results)

In [ ]:
import time
starttime= time.time()

In [ ]:
s

In [ ]:
s.compute()

In [ ]:
endtime = time.time()
print(endtime - starttime)

In [ ]:
dates = list(o['datetime'].dt.date.unique())
k  = [dates[i] for i in range(2)]

In [ ]:
u = df[(df["datetime"] <=price["datetime"][70] ) & (df["datetime"] > (price["datetime"][80] - timedelta(days=70)))]

In [ ]:
type(u)

In [ ]:
s = process_date(k[1],df)
result = dask.compute(s)

In [ ]:
u.compute()

In [ ]:
r = time.time()

t = premiums[(premiums["datetime"] <=price["datetime"][9000] ) & (premiums["datetime"] > (price["datetime"][9000] - timedelta(days=70)))]
t
f = time.time()
print(f-r)

In [ ]:
delayed_results = [process_date(date, df) for date in k]

In [ ]:
bag = db.from_delayed(delayed_results)
results = bag.compute()

In [ ]:
u = []

In [ ]:
u
    

In [ ]:
endtime = time.time()
print(-starttime + endtime)

In [ ]:
import pygame

pygame.mixer.init()
pygame.mixer.music.load(r"C:\Users\yaswa\Downloads\Jolly-O-Gymkhana-MassTamilan.so.mp3")
pygame.mixer.music.play()

In [ ]:
pygame.mixer.music.stop()

In [ ]:
pd.DataFrame(D)

In [ ]:
groups = price.groupby(price['datetime'].dt.date)

In [ ]:
group = price[price['datetime'].dt.date == date] 

In [ ]:
k  = [dates[i] for i in range(6)]

In [ ]:
k